In [ ]:
import os, sys, glob
from tqdm import tqdm
import avstack
import avapi
import cv2
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

data_dir = '../data/MOT15/'

MSM = avapi.mot15.MOT15SceneManager(data_dir=data_dir)
MSD = MSM.get_scene_dataset_by_name('ADL-Rundle-6')

## Run Algorithms

In [ ]:
framerate = 30.0
# detector = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
#     model='fasterrcnn', dataset='coco-person', deploy=False)
detector = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
    model='cascade_rcnn', dataset='coco', deploy=True)
tracker = avstack.modules.tracking.tracker2d.SortTracker2D()

In [ ]:
all_dets = {}
all_tracks = {}
i_frame_start = 390
n_frames = 30; n_frames = min(len(MSD), n_frames)
cvt_to_rgb = True
ego_ref = MSD.get_ego_reference(frame=1)

all_frames = []
for i_frame, frame in tqdm(enumerate(MSD.frames[i_frame_start:i_frame_start+n_frames]), total=n_frames):
    all_frames.append(frame)
    # -- load image in avstack standard format
    img = MSD.get_image(frame=frame)
    
    # -- run detections and tracking
    dets = detector(img)
    tracks = tracker(detections=dets, frame=img.frame, t=img.timestamp, platform=ego_ref)
    
    # -- save results
    all_dets[frame] = dets
    all_tracks[frame] = tracks

In [ ]:
idx_frame = 10
frame = all_frames[idx_frame]
img = MSD.get_image(frame=frame)
objs = MSD.get_objects(frame=frame)
avapi.visualize.snapshot.show_image_with_boxes(img, objs, inline=True) 
avapi.visualize.snapshot.show_image_with_boxes(img, all_dets[frame], inline=True)
avapi.visualize.snapshot.show_image_with_boxes(img, all_tracks[frame], inline=True) 